<a href="https://colab.research.google.com/github/alexsanderthorne/smartSystems/blob/main/salesman_ga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#[1] Importando pacotes e módulos

import random
import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

In [2]:
#[2] Geração do grafo para o problema do caixeiro

def graphTSP(numCities, minDist, maxDist):
  cities = numpy.zeros((numCities, numCities), dtype = int)
  for i in range(numCities):
    for j in range(numCities):
      if (j>i):
        cities[i, j] = random.randint(minDist, maxDist)
      elif (j<i):
        cities[i, j] = cities[j, i]
  return cities

numCities = 5     #  Número de cidade inicial

while(True):
  numCities = int(input('Digite o número de cidades: '))
  if (numCities > 4):
    break
  else:
    print('O número de cidades deve ser maior que 4!')

cities = graphTSP(numCities, 10, 100)
print('Grafo:\n', cities)

Grafo:
 [[ 0 14 68 17 38 91 31 53]
 [14  0 43 42 98 23 98 43]
 [68 43  0 12 75 86 80 63]
 [17 42 12  0 38 19 72 38]
 [38 98 75 38  0 96 84 97]
 [91 23 86 19 96  0 97 64]
 [31 98 80 72 84 97  0 25]
 [53 43 63 38 97 64 25  0]]


In [3]:
#[3] Definição da Geração dos Indivíduos

creator.create("FitnessMin", base.Fitness, weights=(-1.0,)) # minimizar = peso negativo 
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
# gerador de parâmetros 
toolbox.register("attr_int", random.randint, 0, numCities-1)

# define como os indivíduos/população é gerada
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, numCities)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [4]:
#[4] Função para a avaliação da aptidão

#             * precisa ser uma tupla devido a exigências do pacote DEAP 
def evalRoute(individual):
  cost = 0
  for i in range(1, len(individual)):
    if (individual.count(individual[i])>1):
      cost = cost + 1000000 # penalidade por repetir cidade
    cost = cost + cities[individual[i-1], individual[i]]        
  cost = cost + cities[individual[i],individual[0]]
  return (cost,)

In [5]:
#[5] Processamento do Algoritmo Genético

# Definindo avaliação de aptidão, seleção, cruzamento e mutação 
toolbox.register("evaluate", evalRoute)
toolbox.register("select", tools.selTournament, tournsize=3) # seleção por torneio
toolbox.register("mate", tools.cxOnePoint) # um ponto de cruzamento
toolbox.register("mutate", tools.mutUniformInt, low=0, up=numCities-1, indpb=0.05)

def main():
  print('Execução do algoritmo genético:')

  # random.seed(64)
  NGEN = 100     # número de gerações
  MU = 50        # tamanho da população
  LAMBDA = 100   # número de filhos gerados
  CXPB = 0.7     # probabilidade de cruzamento
  MUTPB = 0.3    # probabilidade de mutação
  
  pop = toolbox.population(n=MU)
  hof = tools.ParetoFront()
  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register("avg", numpy.mean, axis=0)
  stats.register("std", numpy.std, axis=0)
  stats.register("min", numpy.min, axis=0)
  stats.register("max", numpy.max, axis=0)
  
  algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                            halloffame=hof)
      
  print('\nRota:', hof[0],'\nCusto:', evalRoute(hof[0])[0])
  return pop, stats, hof
                 
if __name__ == "__main__":
    main()

Execução do algoritmo genético:
gen	nevals	avg         	std               	min       	max       
0  	50    	[4120411.44]	[1394836.07108727]	[1000387.]	[7000529.]
1  	100   	[2660416.44]	[950977.42783141] 	[1000387.]	[5000333.]
2  	100   	[2120401.58]	[790924.01082845] 	[1000333.]	[4000414.]
3  	100   	[1700404.66]	[806176.66078989] 	[483.]    	[3000513.]
4  	100   	[1280443.36]	[664456.20893963] 	[483.]    	[3000432.]
5  	100   	[900464.7]  	[574424.92062508] 	[483.]    	[3000208.]
6  	100   	[520465.5]  	[574078.95429766] 	[483.]    	[2000334.]
7  	100   	[180477.88] 	[384167.47116504] 	[483.]    	[1000678.]
8  	100   	[20484.6]   	[139994.6290324]  	[483.]    	[1000447.]
9  	100   	[483.]      	[0.]              	[483.]    	[483.]    
10 	100   	[483.]      	[0.]              	[483.]    	[483.]    
11 	100   	[483.]      	[0.]              	[483.]    	[483.]    
12 	100   	[483.]      	[0.]              	[483.]    	[483.]    
13 	100   	[483.]      	[0.]              	[483.]    	[483